---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [16]:
pd.read_csv('test.csv', encoding="ISO-8859-1")[['ticket_id', 'agency_name', 'state', 'disposition', 'fine_amount', 'discount_amount', 
                   'judgment_amount']].isnull().sum()

ticket_id            0
agency_name          0
state              331
disposition          0
fine_amount          0
discount_amount      0
judgment_amount      0
dtype: int64

In [22]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


def blight_model():
    train = pd.read_csv('train.csv', encoding='ISO-8859-1')
    test = pd.read_csv('test.csv', encoding="ISO-8859-1")

    #drop rows
    train.drop(train[train.country != 'USA'].index, inplace=True)
    train.dropna(subset=['compliance', 'state'], inplace=True)
    
    X_train = train[['agency_name', 'state', 'disposition', 'fine_amount', 'discount_amount', 'judgment_amount']]
    y_train = train['compliance']
    X_test = test[['ticket_id', 'agency_name', 'state', 'disposition', 'fine_amount', 'discount_amount', 
                   'judgment_amount']]
    X_test.set_index('ticket_id', inplace=True)
    
    X_test.loc['state'] = X_test.loc[:,'state'].fillna(value='MI')
    
    #Create dummy cols
    X_train = pd.get_dummies(X_train, prefix=['dispo', 'a_name', 'ST'], 
                             columns=['disposition', 'agency_name', 'state'], drop_first=True)
    X_test = pd.get_dummies(X_test, prefix=['dispo', 'a_name', 'ST'], 
                            columns=['disposition', 'agency_name', 'state'], drop_first=True)
    
    #The test set produces differente dummy columns (train:67, test:69). 
    #There must be some value for a categorical column missing in the train. Let's fix it
    X_train, X_test = X_train.align(X_test, join='right', axis=1, fill_value=0)

    #Model
#     grid_vals = {'n_estimators': [6, 8, 10, 12,
#                                   14, 16], 'max_depth': [3, 4, 5, 6, 10]}
    model = RandomForestClassifier(n_estimators=16, max_depth=10).fit(X_train, y_train)
#     grid_search = GridSearchCV(model, grid_vals, scoring='roc_auc', n_jobs=-1, cv=5)
#     grid_result = grid_search.fit(X_train, y_train)
    X_test_pred = model.predict_proba(X_test)
    ans = pd.Series(index=X_test.index, data=X_test_pred[:, 1], name='compliance', dtype='float32')

    return ans

In [24]:
# import numpy as np
# bm = blight_model()
# res = '{:40s}'.format('Object Type:')
# res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
# res += '{:40s}'.format('Data Shape:')
# res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
# res += '{:40s}'.format('Data Values Type:')
# res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
# res += '{:40s}'.format('Data Values Infinity:')
# res += ['Failed: values should not be infinity\n','Passed\n'][not any(np.isinf(bm))]
# res += '{:40s}'.format('Data Values NaN:')
# res += ['Failed: values should not be NaN\n','Passed\n'][not any(np.isnan(bm))]
# res += '{:40s}'.format('Data Values in [0,1] Range:')
# res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
# res += '{:40s}'.format('Data Values not all 0 or 1:')
# res += ['Failed: values should be scores not predicted labels\n','Passed\n'][not all((bm.isin({0,1,0.0,1.0})))]
# res += '{:40s}'.format('Index Type:')
# res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
# res += '{:40s}'.format('Index Values:')
# if bm.index.shape==(61001,):
#     res +=['Failed: index values should match test.csv\n','Passed\n'
#           ][all(pd.read_csv('test.csv',usecols=[0],index_col=0
#                            ).sort_index().index.values==bm.sort_index().index.values)]
# else:
#     res+='Failed: bm.index length should be 61001'
# res += '{:40s}'.format('Can run model twice:')
# bm2 = None
# try:
#     bm2 = blight_model()
#     res += 'Passed\n'
# except:
#     res += ['Failed: second run of blight_model() threw an Exception']
# res += '{:40s}'.format('Can run model twice with same results:')
# if not bm2 is None:
#     res += ['Failed: second run of blight_model() produced different results (this might not be a problem)\n','Passed\n'][
#         all(bm.apply(lambda x:round(x,3))==bm2.apply(lambda x:round(x,3))) and all(bm.index==bm2.index)]    
# print(res)

In [23]:
blight_model()

/Users/elece/miniconda3/envs/baseML/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3242: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/elece/miniconda3/envs/baseML/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


ticket_id
284932    0.941985
285362    0.979286
285361    0.940167
285338    0.941985
285346    0.940167
            ...   
376496    0.976616
376497    0.976616
376499    0.946754
376500    0.946754
369851    0.687207
Name: compliance, Length: 61001, dtype: float32